# Boltzmann Machines

In [89]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [90]:
print(torch.__version__)

2.5.1


## Importing Dataset

In [91]:
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1")
users = pd.read_csv("ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1")
# index::userid::moviesid::rating::timestamp
ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, engine="python", encoding="latin-1")

## Preparing the training set and test set

In [92]:
training_set = pd.read_csv("ml-100k/u1.base", delimiter='\t')
training_set = np.array(training_set, dtype="int")

test_set = pd.read_csv("ml-100k/u1.test", delimiter='\t')
test_set = np.array(test_set, dtype="int")

## Getting the number of users and movies

In [93]:
nb_users = int(max(max(training_set[:, 0]), max(training_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(training_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns

In [94]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [95]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Disliked)

In [96]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network

In [97]:
class RBM:
    def __init__(self, nv, nh):
        self.W = torch.randn(nv, nh)  # Weights: (nv, nh)
        self.a = torch.randn(1, nh)   # Hidden bias: (1, nh)
        self.b = torch.randn(1, nv)   # Visible bias: (1, nv)

    def sample_h(self, x):
        wx = torch.mm(x, self.W)  # x: (batch_size, nv), W: (nv, nh)
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W.t())  # y: (batch_size, nh), W.t(): (nh, nv)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk, lr=0.01):
        self.W += lr * (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk))
        self.b += lr * torch.sum((v0 - vk), 0)
        self.a += lr * torch.sum((ph0 - phk), 0)

In [98]:
nv = len(training_set[0])
# number of features
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## Training RBM

In [99]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        # Get a batch of visible units
        v0 = training_set[id_user:id_user + batch_size]  # Shape: (batch_size, n_visible)
        vk = v0.clone()  # Initialize vk with v0

        # Sample hidden units for v0
        ph0, _ = rbm.sample_h(v0)  # ph0 shape: (batch_size, n_hidden)

        # Perform CD-k (random walk)
        for k in range(10):
            _, hk = rbm.sample_h(vk)  # hk shape: (batch_size, n_hidden)
            _, vk = rbm.sample_v(hk)  # vk shape: (batch_size, n_visible)
            # Preserve missing ratings
            vk[v0 < 0] = v0[v0 < 0]

        # Sample hidden units for vk
        phk, _ = rbm.sample_h(vk)  # phk shape: (batch_size, n_hidden)

        # Train the RBM
        rbm.train(v0, vk, ph0, phk)

        # Compute training loss (MAE for rated items)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.0

    # Print epoch loss
    print(f"epoch: {epoch}, loss: {train_loss / s}")


epoch: 1, loss: 0.3850482702255249
epoch: 2, loss: 0.3313833475112915
epoch: 3, loss: 0.29904624819755554
epoch: 4, loss: 0.2868254482746124
epoch: 5, loss: 0.2776229977607727
epoch: 6, loss: 0.27519598603248596
epoch: 7, loss: 0.2738665044307709
epoch: 8, loss: 0.266421914100647
epoch: 9, loss: 0.2606787085533142
epoch: 10, loss: 0.2625701129436493


## Testing the RBM

In [100]:
test_loss = 0
s = 0.0
for id_user in range(0, nb_users):
    # Get a batch of visible units
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    
    # Perform CD-k (random walk)
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)

        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        s += 1.0

# Print loss
print(f"test loss: {train_loss / s}")


test loss: 0.005148433614522219
